***GENERATED CODE FOR datapreep PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run datapreepHooks.ipynb
try:
	#sourcePreExecutionHook()

	botonhouingtrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1632467149469_Boston housing Train.csv', 'filename': '1632467149469_Boston housing Train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/files/Boston housing Train.csv', 'viewFileName': 'Boston housing Train.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://13.59.224.229:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run datapreepHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(botonhouingtrain,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/1632467149469_Boston housing Train.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/1632467149469_Boston housing Train.csv", "data_source": "platfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/61d6af8f416d4d1ddd9f24d21641472300/0part.csv", "requestRatio": 1.0, "totalRows": 506, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 15, "numberOfRows": 506, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "generated": 0, "type": "real", "max": 88.9762, "min": 0.00632, "mean": 3.6135235573122535, "missing": 0.0, "stddev": 8.6}, {"column": "ZN", "alias": "ZN", "generated": 0, "type": "real", "max": 100.0, "min": 0.0, "mean": 11.363636363636363, "missing": 0.0, "stddev": 23.32}, {"column": "INDUS", "alias": "INDUS", "generated": 0, "type": "real", "max": 27.74, "min": 0.46, "mean": 11.136778656126504, "missing": 0.0, "stddev": 6.86}, {"column": "CHAS", "alias": "CHAS", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.0691699604743083, "missing": 0.0, "stddev": 0.25}, {"column": "NOX", "alias": "NOX", "generated": 0, "type": "real", "max": 0.871, "min": 0.385, "mean": 0.5546950592885372, "missing": 0.0, "stddev": 0.12}, {"column": "RM", "alias": "RM", "generated": 0, "type": "real", "max": 8.78, "min": 3.5610000000000004, "mean": 6.284634387351788, "missing": 0.0, "stddev": 0.7}, {"column": "AGE", "alias": "AGE", "generated": 0, "type": "real", "max": 100.0, "min": 2.9, "mean": 68.57490118577078, "missing": 0.0, "stddev": 28.15}, {"column": "DIS", "alias": "DIS", "generated": 0, "type": "real", "max": 12.1265, "min": 1.1296, "mean": 3.795042687747034, "missing": 0.0, "stddev": 2.11}, {"column": "RAD", "alias": "RAD", "generated": 0, "type": "real", "max": 24.0, "min": 1.0, "mean": 9.549407114624506, "missing": 0.0, "stddev": 8.71}, {"column": "TAX", "alias": "TAX", "generated": 0, "type": "real", "max": 711.0, "min": 187.0, "mean": 408.2371541501976, "missing": 0.0, "stddev": 168.54}, {"column": "PTRATIO", "alias": "PTRATIO", "generated": 0, "type": "real", "max": 22.0, "min": 12.6, "mean": 18.455533596837967, "missing": 0.0, "stddev": 2.16}, {"column": "B", "alias": "B", "generated": 0, "type": "real", "max": 396.9, "min": 0.32, "mean": 356.67403162055257, "missing": 0.0, "stddev": 91.29}, {"column": "LSTAT", "alias": "LSTAT", "generated": 0, "type": "real", "max": 37.97, "min": 1.73, "mean": 12.653063241106723, "missing": 0.0, "stddev": 7.14}, {"column": "MEDV", "alias": "MEDV", "generated": 0, "type": "real", "max": 50.0, "min": 5.0, "mean": 22.532806324110698, "missing": 0.0, "stddev": 9.2}, {"column": "DISUPDATED", "alias": "DISUPDATED", "generated": 1, "type": "real", "max": 88.0, "min": 4.09, "mean": 71.59548142292486, "missing": 0.0, "stddev": 33.09}]}, "predictionPowerScore": [{"AGE": 1.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.1819418929, "DIS": 0.3654966571, "DISUPDATED": 0.0955446902, "INDUS": 0.4743910252, "LSTAT": 0.031921721, "MEDV": 0.12910914, "NOX": 0.501954171, "PTRATIO": 0.3398705519, "RAD": 0.160634802, "RM": 0.0, "TAX": 0.4286750281, "ZN": 0.2250916338}, {"AGE": 0.0, "B": 1.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "DISUPDATED": 0.0, "INDUS": 0.0, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.1311605518, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 1.0, "CRIM": 0.0, "DIS": 0.0, "DISUPDATED": 0.0, "INDUS": 0.0, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.0, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 1.0, "DIS": 0.0, "DISUPDATED": 0.0, "INDUS": 0.3943513349, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.4181366721, "PTRATIO": 0.3498679351, "RAD": 0.3439931322, "RM": 0.0, "TAX": 0.3892382262, "ZN": 0.0}, {"AGE": 0.2263840804, "B": 0.0, "CHAS": 0.0, "CRIM": 0.1098016755, "DIS": 1.0, "DISUPDATED": 0.1667427523, "INDUS": 0.7192599348, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.7685256976, "PTRATIO": 0.4735583747, "RAD": 0.2010614961, "RM": 0.0, "TAX": 0.6826358869, "ZN": 0.3440131232}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.9898185941, "DISUPDATED": 1.0, "INDUS": 0.3964738673, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.3595007455, "PTRATIO": 0.0898559195, "RAD": 0.0, "RM": 0.0, "TAX": 0.2776000262, "ZN": 0.0}, {"AGE": 0.2109846408, "B": 0.007361998, "CHAS": 0.0, "CRIM": 0.4290394368, "DIS": 0.4042352387, "DISUPDATED": 0.06115494, "INDUS": 1.0, "LSTAT": 0.0617684759, "MEDV": 0.2447842529, "NOX": 0.9094313843, "PTRATIO": 0.6640834779, "RAD": 0.4417218805, "RM": 0.0, "TAX": 0.8713858002, "ZN": 0.2662979852}, {"AGE": 0.062603761, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0983268519, "DISUPDATED": 0.0248212188, "INDUS": 0.3050010509, "LSTAT": 1.0, "MEDV": 0.336130308, "NOX": 0.3489632496, "PTRATIO": 0.1996542817, "RAD": 0.1740047184, "RM": 0.0872401293, "TAX": 0.2588577211, "ZN": 0.130676928}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "DISUPDATED": 0.0, "INDUS": 0.2739922522, "LSTAT": 0.192382389, "MEDV": 1.0, "NOX": 0.3534215648, "PTRATIO": 0.1830365575, "RAD": 0.0822970281, "RM": 0.134605752, "TAX": 0.1831393252, "ZN": 0.0969028871}, {"AGE": 0.2214231031, "B": 0.0, "CHAS": 0.0, "CRIM": 0.3529646943, "DIS": 0.3969063132, "DISUPDATED": 0.10790784, "INDUS": 0.7475713158, "LSTAT": 0.0, "MEDV": 0.1373945132, "NOX": 1.0, "PTRATIO": 0.523485966, "RAD": 0.3203099503, "RM": 0.0, "TAX": 0.6772530451, "ZN": 0.2350807327}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0673137855, "DIS": 0.0907940989, "DISUPDATED": 0.0154578751, "INDUS": 0.9172705087, "LSTAT": 0.0, "MEDV": 0.0215388598, "NOX": 0.8518213838, "PTRATIO": 1.0, "RAD": 0.3315092837, "RM": 0.0, "TAX": 0.7655938121, "ZN": 0.1993584355}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.7525769844, "DIS": 0.0, "DISUPDATED": 0.0, "INDUS": 0.9783928152, "LSTAT": 0.0, "MEDV": 0.0667400174, "NOX": 0.9687788517, "PTRATIO": 0.8051119616, "RAD": 1.0, "RM": 0.0, "TAX": 0.9212987046, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "DISUPDATED": 0.0, "INDUS": 0.1465495983, "LSTAT": 0.0, "MEDV": 0.1446528159, "NOX": 0.1310865934, "PTRATIO": 0.0976885308, "RAD": 0.005901464, "RM": 1.0, "TAX": 0.0524240323, "ZN": 0.0894627131}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.5597919875, "DIS": 0.1597987317, "DISUPDATED": 0.0078472622, "INDUS": 0.96937539, "LSTAT": 0.0, "MEDV": 0.1921251742, "NOX": 0.9474391316, "PTRATIO": 0.7269644848, "RAD": 0.6943762355, "RM": 0.0, "TAX": 1.0, "ZN": 0.075200725}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.4205587427, "DISUPDATED": 0.0, "INDUS": 0.8555653668, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.7167584404, "PTRATIO": 0.3032268596, "RAD": 0.0, "RM": 0.0, "TAX": 0.5283745907, "ZN": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "DIS", "type": "real", "min": 1.1, "max": 12.1, "mean": 3.8}], "functionsData": [{"replaceType": "value", "toReplace": "4.09", "replaceValueType": "byValue", "replaceValueCondition": "lessthan", "ReplaceWith": "66", "asNewColumn": 1, "newColumnName": "DISUPDATED"}, {"replaceType": "value", "toReplace": "6.0622", "replaceValueType": "byValue", "replaceValueCondition": "greaterthan", "ReplaceWith": "88", "asNewColumn": 1, "newColumnName": "DISUPDATED"}]}], "functionChanges": [{"columnName": "DIS", "functionName": "Replace Outliers", "Type": "real", "Parameters": [{"replaceType": "value", "toReplace": "4.09", "replaceValueType": "byValue", "replaceValueCondition": "lessthan", "ReplaceWith": "66", "asNewColumn": 1, "newColumnName": "DISUPDATED"}, {"replaceType": "value", "toReplace": "6.0622", "replaceValueType": "byValue", "replaceValueCondition": "greaterthan", "ReplaceWith": "88", "asNewColumn": 1, "newColumnName": "DISUPDATED"}]}], "fileheader": [{"field": "CRIM", "alias": "CRIM", "generated": 0, "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "generated": 0, "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "generated": 0, "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "generated": 0, "position": 4, "type": "real"}, {"field": "NOX", "alias": "NOX", "generated": 0, "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "generated": 0, "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "generated": 0, "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "generated": 0, "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "generated": 0, "position": 9, "type": "real"}, {"field": "TAX", "alias": "TAX", "generated": 0, "position": 10, "type": "real"}, {"field": "PTRATIO", "alias": "PTRATIO", "generated": 0, "position": 11, "type": "real"}, {"field": "B", "alias": "B", "generated": 0, "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "generated": 0, "position": 13, "type": "real"}, {"field": "MEDV", "alias": "MEDV", "generated": 0, "position": 14, "type": "real"}, {"field": "DISUPDATED", "alias": "DISUPDATED", "generated": 1, "position": 15, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run datapreepHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "2.33", "stddev": "5.05", "min": "0.01432", "max": "22.5971", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.3", "stddev": "21.95", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "9.55", "stddev": "5.79", "min": "1.32", "max": "19.58", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.04", "stddev": "0.2", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.54", "stddev": "0.1", "min": "0.41100000000000003", "max": "0.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "6.48", "stddev": "0.77", "min": "5.0", "max": "8.725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "72.41", "stddev": "24.8", "min": "17.2", "max": "98.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "3.76", "stddev": "2.09", "min": "1.5184", "max": "8.3248", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.96", "stddev": "7.85", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "399.52", "stddev": "149.09", "min": "193.0", "max": "666.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "18.38", "stddev": "1.99", "min": "14.7", "max": "21.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "340.71", "stddev": "114.3", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "11.52", "stddev": "6.86", "min": "2.98", "max": "31.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "22.64", "stddev": "10.65", "min": "7.4", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DISUPDATED", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "74.73", "stddev": "31.03", "min": "4.2673", "max": "88.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DISUPDATED"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run datapreepHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "DISUPDATED"], "MEDV")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

